### Scrapping-Project

In [1]:
#importing libraries
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
def get_headers(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content)
    #all the header tag will belong to either one of the following
    headings = ['h1','h2','h3','h4','h5','h6']
    headers = soup.find_all(headings)
    for h in headers:
        print(h)
#         print(h.text) #If we want content of headers
    

In [3]:
get_headers("https://en.wikipedia.org/wiki/Main_Page")

<h1 class="firstHeading" id="firstHeading">Main Page</h1>
<h2 class="mp-h2" id="mp-tfa-h2"><span id="From_today.27s_featured_article"></span><span class="mw-headline" id="From_today's_featured_article">From today's featured article</span></h2>
<h2 class="mp-h2" id="mp-dyk-h2"><span class="mw-headline" id="Did_you_know_...">Did you know ...</span></h2>
<h2 class="mp-h2" id="mp-itn-h2"><span class="mw-headline" id="In_the_news">In the news</span></h2>
<h2 class="mp-h2" id="mp-otd-h2"><span class="mw-headline" id="On_this_day">On this day</span></h2>
<h2 class="mp-h2" id="mp-tfp-h2"><span id="Today.27s_featured_picture"></span><span class="mw-headline" id="Today's_featured_picture">Today's featured picture</span></h2>
<h2 class="mp-h2" id="mp-other"><span class="mw-headline" id="Other_areas_of_Wikipedia">Other areas of Wikipedia</span></h2>
<h2 class="mp-h2" id="mp-sister"><span id="Wikipedia.27s_sister_projects"></span><span class="mw-headline" id="Wikipedia's_sister_projects">Wikipedia'

In [4]:
def get_imdb(url, indian=False):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    rows = soup.find_all('tr')
    if indian:
        f_name = "imdb_100_indian.csv"
    else:
        f_name = "imdb_top_100.csv"
    with open(f_name, "w", encoding="utf-8") as movies:
        movies.write("Name,Year,Rating\n")
        for row in rows[1:101]:
            title = row.find("td", {"class": "titleColumn"}).text
            splt = title.split("(")
            name  = splt[0].split(".")[1].strip()
            year = splt[1].split(")")[0]
            rat = row.find("td", {"class": "imdbRating"}).text.strip()
            movies.write(name + "," + year + "," +  rat + '\n')

In [5]:
get_imdb('http://www.imdb.com/chart/top')

In [6]:
get_imdb('https://www.imdb.com/india/top-rated-indian-movies/', True)

In [7]:
def get_books(url):
    response = requests.get(url + "/reviews")

    soup = BeautifulSoup(response.content, 'lxml')
    rows = soup.find_all("div", {"class":["bp-block", "article-info"]})
    for row, index in zip(rows[0:5], range(1,6)):
        review_link = url + row.find('h4').a.get("href")
        detail_resp = requests.get(review_link)
        detail_soup = BeautifulSoup(detail_resp.content, "lxml")
        title = detail_soup.find("h1").text.strip()
        author = detail_soup.find("h4").text.strip()
        genre = row.find("p", {"class":"genre-links"}).text.strip().replace("/", ",")
        review =  detail_soup.find("div", {"class":"article-body"})
        review = review.find_all("p")
        review_text = ' '.join([str(elem.text) for elem in review])
        print(f"""
{index}. {title}
author: {author}
genre: {genre}
review: {review_text}
-------END {index}--------
""")

In [8]:
get_books("https://bookpage.com")


1. Tokyo Ever After
author: Emiko Jean
genre: YA
 , 
YA Fiction
review: 
 There’s no shortage of YA novels in which a commoner gets involved with a royal family and/or discovers their royal lineage, but as enjoyable as this trope is, most of these novels involve British or European monarchies. Emiko Jean’s Tokyo Ever After sets a thoroughly modern fairy tale in the Imperial House of Japan. Seventeen-year-old Izumi Tanaka has never felt like she completely belongs in her insular, mostly white Northern California town. Sure, she loves and admires her single mom, and she absolutely adores her small group of pan-Asian friends, who’ve dubbed themselves the “Asian Girl Gang,” or AGG for short. (“Think less organized crime, more ‘Golden Girls.’”). But unlike her friends, Izumi has no clear idea what’s next after senior year or why she feels so adrift. Then Izumi’s friend Noora discovers an evocative love note hidden in one of Izumi’s mom’s books, dated with Izumi’s birth year, and traces it 

In [9]:
def ten_odis(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    rows = soup.find("tbody").find_all("tr")
    for row in rows[0:10]:
        info = row.find_all("td")
        print(f"""
S.No:    {info[0].text}.
Country: {info[1].find_all("span")[1].text}
Matches: {info[2].text}
Points:  {info[3].text}
Rating:  {info[4].text.strip()}
-------END {info[0].text}--------
""")


In [10]:
ten_odis("https://www.icc-cricket.com/rankings/mens/team-rankings/odi")


S.No:    1.
Country: New Zealand
Matches: 17
Points:  2,054
Rating:  121
-------END 1--------


S.No:    2.
Country: Australia
Matches: 25
Points:  2,945
Rating:  118
-------END 2--------


S.No:    3.
Country: India
Matches: 29
Points:  3,344
Rating:  115
-------END 3--------


S.No:    4.
Country: England
Matches: 27
Points:  3,100
Rating:  115
-------END 4--------


S.No:    5.
Country: South Africa
Matches: 20
Points:  2,137
Rating:  107
-------END 5--------


S.No:    6.
Country: Pakistan
Matches: 24
Points:  2,323
Rating:  97
-------END 6--------


S.No:    7.
Country: Bangladesh
Matches: 24
Points:  2,157
Rating:  90
-------END 7--------


S.No:    8.
Country: West Indies
Matches: 27
Points:  2,222
Rating:  82
-------END 8--------


S.No:    9.
Country: Sri Lanka
Matches: 21
Points:  1,652
Rating:  79
-------END 9--------


S.No:    10.
Country: Afghanistan
Matches: 17
Points:  1,054
Rating:  62
-------END 10--------



In [11]:
def m_player_odis(url, ball=0):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    div = soup.find_all("div", {"class":"col-4"})
    rows = div[ball]
    first_name = rows.find("div", {"class": "rankings-block__banner--name"}).text
    first_country = rows.find("div", {"class": "rankings-block__banner--nationality"}).text.strip()[0:3]
    first_points = rows.find("div", {"class": "rankings-block__banner--rating"}).text.strip()
    print(f"""
    Pos:    1
    Player: {first_name}
    Team: {first_country}
    Rating:  {first_points}
    ------- END 1 --------
    """)
    trs = rows.find_all("tr")
    for tr in trs[1:10]:
        pos = tr.find("td", {"class":"table-body__cell table-body__cell--position u-text-right"}).text.strip()
        player = tr.find("td", {"class":["name"]}).text.strip()
        team = tr.find("span", {"class":"table-body__logo-text"}).text.strip()
        rating = tr.find("td", {"class":"table-body__cell u-text-right rating"}).text.strip()
        print(f"""
            Pos:    {pos.split(" ")[0].strip()}
            Player: {player}
            Team: {team}
            Rating:  {rating}
            ------- END {pos.split(" ")[0].strip()} --------
            """)

In [12]:
m_player_odis("https://www.icc-cricket.com/rankings/mens/player-rankings/odi")


    Pos:    1
    Player: Babar Azam
    Team: PAK
    Rating:  865
    ------- END 1 --------
    

            Pos:    2
            Player: Virat Kohli
            Team: IND
            Rating:  857
            ------- END 2 --------
            

            Pos:    3
            Player: Rohit Sharma
            Team: IND
            Rating:  825
            ------- END 3 --------
            

            Pos:    4
            Player: Ross Taylor
            Team: NZ
            Rating:  801
            ------- END 4 --------
            

            Pos:    5
            Player: Aaron Finch
            Team: AUS
            Rating:  791
            ------- END 5 --------
            

            Pos:    6
            Player: Jonny Bairstow
            Team: ENG
            Rating:  785
            ------- END 6 --------
            

            Pos:    7
            Player: Fakhar Zaman
            Team: PAK
            Rating:  778
            ------- END 7 --------
        

In [13]:
m_player_odis("https://www.icc-cricket.com/rankings/mens/player-rankings/odi", 1)


    Pos:    1
    Player: Trent Boult
    Team: NZ

    Rating:  737
    ------- END 1 --------
    

            Pos:    2
            Player: Mujeeb Ur Rahman
            Team: AFG
            Rating:  708
            ------- END 2 --------
            

            Pos:    3
            Player: Matt Henry
            Team: NZ
            Rating:  691
            ------- END 3 --------
            

            Pos:    4
            Player: Jasprit Bumrah
            Team: IND
            Rating:  690
            ------- END 4 --------
            

            Pos:    5
            Player: Mehedi Hasan
            Team: BAN
            Rating:  668
            ------- END 5 --------
            

            Pos:    6
            Player: Kagiso Rabada
            Team: SA
            Rating:  666
            ------- END 6 --------
            

            Pos:    7
            Player: Chris Woakes
            Team: ENG
            Rating:  665
            ------- END 7 --------
  

In [14]:
ten_odis("https://www.icc-cricket.com/rankings/womens/team-rankings/odi")


S.No:    1.
Country: Australia
Matches: 18
Points:  2,955
Rating:  164
-------END 1--------


S.No:    2.
Country: South Africa
Matches: 24
Points:  2,828
Rating:  118
-------END 2--------


S.No:    3.
Country: England
Matches: 17
Points:  1,993
Rating:  117
-------END 3--------


S.No:    4.
Country: India
Matches: 20
Points:  2,226
Rating:  111
-------END 4--------


S.No:    5.
Country: New Zealand
Matches: 21
Points:  1,947
Rating:  93
-------END 5--------


S.No:    6.
Country: West Indies
Matches: 12
Points:  1,025
Rating:  85
-------END 6--------


S.No:    7.
Country: Pakistan
Matches: 15
Points:  1,101
Rating:  73
-------END 7--------


S.No:    8.
Country: Bangladesh
Matches: 5
Points:  306
Rating:  61
-------END 8--------


S.No:    9.
Country: Sri Lanka
Matches: 11
Points:  519
Rating:  47
-------END 9--------


S.No:    10.
Country: Ireland
Matches: 2
Points:  25
Rating:  13
-------END 10--------



In [15]:
def w_player_odis(url, bat=0):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    rows = soup.find_all("tr")

    first_name = rows[1].find_all("a")[1].text.strip()
    first_country = rows[1].find("div", {"class": "rankings-block__banner--nationality"}).text.strip()
    first_points = rows[1].find("div", {"class": "rankings-block__banner--rating"}).text.strip()
    print(f"""
    Pos:    1
    Player: {first_name}
    Team: {first_country}
    Rating:  {first_points}
    ------- END 1 --------
    """)
    for row in rows[2:11]:
        pos = row.find("td", {"class":"table-body__cell table-body__cell--position u-text-right"}).text.strip()
        player = row.find("td", {"class":["name"]}).a.text.strip()
        team = row.find("span", {"class": "table-body__logo-text"}).text.strip()
        rating = row.find("td", {"class": ["rating"]}).text.strip()
        print(f"""
            Pos:    {pos.split(" ")[0].strip()}
            Player: {player}
            Team: {team}
            Rating:  {rating}
            ------- END {pos.split(" ")[0].strip()} --------
            """)

In [16]:
w_player_odis("https://www.icc-cricket.com/rankings/womens/player-rankings/odi/batting")


    Pos:    1
    Player: Tammy Beaumont
    Team: ENG
    Rating:  765
    ------- END 1 --------
    

            Pos:    2
            Player: Lizelle Lee
            Team: SA
            Rating:  758
            ------- END 2 --------
            

            Pos:    3
            Player: Alyssa Healy
            Team: AUS
            Rating:  756
            ------- END 3 --------
            

            Pos:    4
            Player: Stafanie Taylor
            Team: WI
            Rating:  746
            ------- END 4 --------
            

            Pos:    5
            Player: Meg Lanning
            Team: AUS
            Rating:  723
            ------- END 5 --------
            

            Pos:    6
            Player: Amy Satterthwaite
            Team: NZ
            Rating:  715
            ------- END 6 --------
            

            Pos:    7
            Player: Smriti Mandhana
            Team: IND
            Rating:  710
            ------- END 7 -----

In [17]:
w_player_odis("https://www.icc-cricket.com/rankings/womens/player-rankings/odi/all-rounder")


    Pos:    1
    Player: Marizanne Kapp
    Team: SA
    Rating:  418
    ------- END 1 --------
    

            Pos:    This
            Player: Ellyse Perry
            Team: AUS
            Rating:  418
            ------- END This --------
            

            Pos:    3
            Player: Stafanie Taylor
            Team: WI
            Rating:  410
            ------- END 3 --------
            

            Pos:    4
            Player: Natalie Sciver
            Team: ENG
            Rating:  349
            ------- END 4 --------
            

            Pos:    5
            Player: Deepti Sharma
            Team: IND
            Rating:  343
            ------- END 5 --------
            

            Pos:    6
            Player: Jess Jonassen
            Team: AUS
            Rating:  307
            ------- END 6 --------
            

            Pos:    7
            Player: Ashleigh Gardner
            Team: AUS
            Rating:  252
            ------- EN

In [18]:
def get_products(url):
    headers = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}
#     proxies_list = ["128.199.109.241:8080", "113.53.230.195:3128", "125.141.200.53:80", "125.141.200.14:80",
#                     "128.199.200.112:138", "149.56.123.99:3128", "128.199.200.112:80", "125.141.200.39:80",
#                     "134.213.29.202:4444"]
#     proxies = {'https': random.choice(proxies_list)}
    response = requests.post(url,headers=headers)
    soup = BeautifulSoup(response.content, "lxml")
    rows = soup.find_all("div", {"class":"s-include-content-margin"})
    for row in rows:
        data = row.find_all("div", {"class": "sg-row"})[1]
        product = data.find("span", {"class": ["a-size-medium", "a-color-base", "a-text-normal"]}).text
        try:
            price = data.find("span", {"class": ["a-price-whole"]}).text
        except:
            price = "Available from different sellers"
        image = data.find("img", {"class": ["s-image"]}).get("src")
        try:
            rating = data.find("i", {"class": ["a-icon", "a-icon-star", "a-star-4"]}).span.text.split()[0]
        except:
            rating = "No rating"
        print(f"""
Product: {product}
Price: {price}
Image URL: {image}
Rating(out of 5): {rating}
------- END PRODUCT ----------
""")
    next = soup.find("li", {"class": "a-last"})
    if next != None:
        next_url = "https://www.amazon.in" + next.a.get("href")
        get_products(next_url)


get_products("https://www.amazon.in/s?k=mobile+phones+under+20000&rh=n%3A1389432031&ref=nb_sb_noss")


Product: Redmi 9 (Sky Blue, 4GB RAM, 64GB Storage)| 5000 mAh| 2.3GHz Mediatek Helio G35 Octa core Processor
Price: 8,799
Image URL: https://m.media-amazon.com/images/I/71A9Vo1BatL._AC_UY218_.jpg
Rating(out of 5): 4.2
------- END PRODUCT ----------


Product: Samsung Galaxy M31 (Ocean Blue, 6GB RAM, 128GB Storage)
Price: 14,999
Image URL: https://m.media-amazon.com/images/I/71-Su4Wr0HL._AC_UY218_.jpg
Rating(out of 5): 4.3
------- END PRODUCT ----------


Product: Redmi 9A (Nature Green, 2GB RAM, 32GB Storage) | 2GHz Octa-core Helio G25 Processor | 5000 mAh Battery
Price: 6,799
Image URL: https://m.media-amazon.com/images/I/71sxlhYhKWL._AC_UY218_.jpg
Rating(out of 5): 4.2
------- END PRODUCT ----------


Product: Redmi Note 10 (Shadow Black, 6GB RAM, 128GB Storage)
Price: 14,499
Image URL: https://m.media-amazon.com/images/I/715t-uJnjMS._AC_UY218_.jpg
Rating(out of 5): 4.0
------- END PRODUCT ----------


Product: Oppo A31 (Fantasy White, 6GB RAM, 128GB Storage) with No Cost EMI/Additio


Product: Vivo Y91i (Ocean Blue, 2GB RAM, 32GB Storage) with No Cost EMI/Additional Exchange Offers
Price: 7,490
Image URL: https://m.media-amazon.com/images/I/51FnmxwFHDL._AC_UY218_.jpg
Rating(out of 5): 4.1
------- END PRODUCT ----------


Product: Samsung Galaxy M11 (Metallic Blue, 4GB RAM, 64GB Storage) with No Cost EMI/Additional Exchange Offers
Price: 8,999
Image URL: https://m.media-amazon.com/images/I/71GQUxuSpnS._AC_UY218_.jpg
Rating(out of 5): 4.2
------- END PRODUCT ----------


Product: Redmi Note 9 (Pebble Grey, 4GB RAM 64GB Storage) - 48MP Quad Camera & Full HD+ Display
Price: 10,999
Image URL: https://m.media-amazon.com/images/I/71X5I1cVfbL._AC_UY218_.jpg
Rating(out of 5): 4.3
------- END PRODUCT ----------


Product: TECNO Spark 7 (Spruce Green, 3GB RAM, 64 GB Storage) - 6000mAh Battery|16 MP Dual Camera| 6.52” Dot Notch Display
Price: 8,499
Image URL: https://m.media-amazon.com/images/I/719b6ihbwZS._AC_UY218_.jpg
Rating(out of 5): 3.9
------- END PRODUCT ----------


P


Product: Redmi 9A (Nature Green, 2GB RAM, 32GB Storage) | 2GHz Octa-core Helio G25 Processor | 5000 mAh Battery
Price: 7,499
Image URL: https://m.media-amazon.com/images/I/71sxlhYhKWL._AC_UY218_.jpg
Rating(out of 5): 4.2
------- END PRODUCT ----------


Product: Cell Phone, Ultra‑Thin Sim Card Dual Card Double Standby Mini Phone V66+ for School(Blue)
Price: Available from different sellers
Image URL: https://m.media-amazon.com/images/I/51mwKu0iPJL._AC_UY218_.jpg
Rating(out of 5): No rating
------- END PRODUCT ----------


Product: Infinix Hot 9 Pro (Quetzal Cyan, 4GB RAM, 64GB Storage)
Price: 10,999
Image URL: https://m.media-amazon.com/images/I/71wTPrC8N3L._AC_UY218_.jpg
Rating(out of 5): No rating
------- END PRODUCT ----------


Product: Unlocked Cell Phone, Bang Screen 1+8G HD Dual Cards Dual Standby Cell Phone All‑in‑one 6.26in with Beauty Camera(European regulations)
Price: Available from different sellers
Image URL: https://m.media-amazon.com/images/I/61jrMVEVSaL._AC_UY218_.jpg


Product: New Jio Phone 3 (2021) Security Deposit (Black)
Price: 1,899
Image URL: https://m.media-amazon.com/images/I/41vAKCTcs+S._AC_UY218_.jpg
Rating(out of 5): 1.9
------- END PRODUCT ----------


Product: I KALL K1 Smartphone (1GB, 8GB, 4G Volte) (Black)
Price: Available from different sellers
Image URL: https://m.media-amazon.com/images/I/61EusLkWbdL._AC_UY218_.jpg
Rating(out of 5): No rating
------- END PRODUCT ----------


Product: Samsung Metro 313 (SM-B313E, Gold)
Price: 2,129
Image URL: https://m.media-amazon.com/images/I/71MNgRR8kWL._AC_UY218_.jpg
Rating(out of 5): 4.1
------- END PRODUCT ----------


Product: Mini Phone, Cell Phone Dual Card Slot V66+ for School(Blue)
Price: Available from different sellers
Image URL: https://m.media-amazon.com/images/I/51mwKu0iPJL._AC_UY218_.jpg
Rating(out of 5): No rating
------- END PRODUCT ----------


Product: I KALL K480 Smartphone (Blue, 5.5 Inch, 2GB, 16GB)
Price: Available from different sellers
Image URL: https://m.media-amazon.co


Product: Samsung Galaxy A12 (Blue,4GB RAM, 128GB Storage) with No Cost EMI/Additional Exchange Offers
Price: 13,999
Image URL: https://m.media-amazon.com/images/I/91te-9mP2JL._AC_UY218_.jpg
Rating(out of 5): 4.0
------- END PRODUCT ----------


Product: Samsung Guru FM Plus (SM-B110E/D, Black)
Price: 1,449
Image URL: https://m.media-amazon.com/images/I/71X7mmH7fgL._AC_UY218_.jpg
Rating(out of 5): 4.2
------- END PRODUCT ----------


Product: Kechaoda K33 Slim Card Size Light Weight and Stylish Dual Sim GSM Mobile Phone (Rose Gold)
Price: Available from different sellers
Image URL: https://m.media-amazon.com/images/I/81Orn2dJswL._AC_UY218_.jpg
Rating(out of 5): 3.7
------- END PRODUCT ----------


Product: Realme C12 (Power Blue, 3GB RAM, 32GB Storage) Without Offer
Price: 8,699
Image URL: https://m.media-amazon.com/images/I/31X1+mh8zPL._AC_UY218_.jpg
Rating(out of 5): 4.1
------- END PRODUCT ----------


Product: I KALL K560 Smartphone (6.53 HD+, 4GB, 64GB) (Purple)
Price: Available f

...


Product: Nillkin Case for Samsung Galaxy S21 S 21 (6.2" Inch) CamShield Pro Slider Camera Close & Open Double Layered Protection TPU + PC Blue Color
Price: 990
Image URL: https://m.media-amazon.com/images/I/713rqBUlJrL._AC_UY218_.jpg
Rating(out of 5): No rating
------- END PRODUCT ----------


Product: Kechaoda K60 (Golden)
Price: 1,426
Image URL: https://m.media-amazon.com/images/I/41qSG70O6GL._AC_UY218_.jpg
Rating(out of 5): 3.6
------- END PRODUCT ----------


Product: Vivo Y20 (Obsidian Black, 6GB RAM, 64GB Storage) with Infinity (JBL) Wireless Neckband Earphones
Price: 15,190
Image URL: https://m.media-amazon.com/images/I/61l9OCm-u3L._AC_UY218_.jpg
Rating(out of 5): 4.3
------- END PRODUCT ----------


Product: Fonokase Case for Apple iPhone 7 Plus Signature Leather Wrap Series Exclusive Limited - Blue & Brown
Price: 849
Image URL: https://m.media-amazon.com/images/I/71cdoQ40yLL._AC_UY218_.jpg
Rating(out of 5): No rating
------- END PRODUCT ----------


Product: G'Five i2 || Dual


Product: Clout X425 Shavit
Price: Available from different sellers
Image URL: https://m.media-amazon.com/images/I/41OOtb2XIXL._AC_UY218_.jpg
Rating(out of 5): No rating
------- END PRODUCT ----------


Product: Nillkin Case for Xiaomi Redmi Note 8 (6.3" Inch) Super Frosted Hard Back Cover PC Black Color
Price: 899
Image URL: https://m.media-amazon.com/images/I/41YVLpJHZkL._AC_UY218_.jpg
Rating(out of 5): No rating
------- END PRODUCT ----------


Product: Jivi JV 12M (Black)
Price: Available from different sellers
Image URL: https://m.media-amazon.com/images/I/61gCZ4cXKqL._AC_UY218_.jpg
Rating(out of 5): 3.3
------- END PRODUCT ----------


Product: Nillkin Tempered Glass for Samsung Galaxy J7 Duo Amazing H Explosion Proof Screen Protect + Camera Film
Price: 499
Image URL: https://m.media-amazon.com/images/I/812QOnPGrNL._AC_UY218_.jpg
Rating(out of 5): 5.0
------- END PRODUCT ----------


Product: CROGIE Best Pubg Anti-Slip Thumb Sleeve, Slip-Bestof Sweat-Bestof Bestfessional Touch Sc


Product: Nillkin Case for Apple iPhone 11 Pro Max (6.5" Inch) CamShield Armor Military Grade Camera Close & Open Double Layered Protection TPU + PC Finish with Kickstand Dark Green
Price: 3,499
Image URL: https://m.media-amazon.com/images/I/41cdw1P-XWL._AC_UY218_.jpg
Rating(out of 5): No rating
------- END PRODUCT ----------


Product: Samsung Guru FM E1220 (Black)
Price: Available from different sellers
Image URL: https://m.media-amazon.com/images/I/51LgvTdaZnL._AC_UY218_.jpg
Rating(out of 5): No rating
------- END PRODUCT ----------


Product: Nillkin Case for Apple iPhone 12 / Apple iPhone 12 Pro (6.1" Inch) CamShield Pro Camera Close & Open Double Layered Protection TPU + PC Black Color
Price: Available from different sellers
Image URL: https://m.media-amazon.com/images/I/51gVMQqjpWL._AC_UY218_.jpg
Rating(out of 5): No rating
------- END PRODUCT ----------


Product: Canbee A3310 Multimedia Dual Sim Keypad Phone with Camera, Fm, Calculator, Stopwatch, Calendar and Sd Card Support



Product: Nillkin Case for Xiaomi Mi 10i (6.67" Inch) Nature Series Back Soft Flexible TPU Grey Color
Price: 749
Image URL: https://m.media-amazon.com/images/I/61ybbCvC9cL._AC_UY218_.jpg
Rating(out of 5): No rating
------- END PRODUCT ----------


Product: MobiSpare® 100% Orignal Replacment Back Panel/Back Housing/Back Door/Back Body Compatible for Redmi 3s Prime (Grey)
Price: 799
Image URL: https://m.media-amazon.com/images/I/21K9UgNMfCL._AC_UY218_.jpg
Rating(out of 5): No rating
------- END PRODUCT ----------


Product: Lava KKT Pearl (Black & Gold)
Price: Available from different sellers
Image URL: https://m.media-amazon.com/images/I/31mifMOoc2L._AC_UY218_.jpg
Rating(out of 5): 2.6
------- END PRODUCT ----------


Product: SNOWTEL S-90 Mobile with Charger and Battery (Blue)
Price: Available from different sellers
Image URL: https://m.media-amazon.com/images/I/41g5l-gUeML._AC_UY218_.jpg
Rating(out of 5): No rating
------- END PRODUCT ----------


Product: Nillkin Galaxy S21 Case - Ca


Product: Nillkin Case for Samsung Galaxy M31s M 31s (6.5" Inch) Super Frosted Hard Back Cover PC Peacock Blue Color
Price: 999
Image URL: https://m.media-amazon.com/images/I/51H+YSB5AiL._AC_UY218_.jpg
Rating(out of 5): No rating
------- END PRODUCT ----------


Product: Nillkin Screen Guard for Huawei Honor 9X 9 X Pro (6.59" Inch) Whole Set Version Super Clear Front + Camera Sticker
Price: 399
Image URL: https://m.media-amazon.com/images/I/41pK1BSBsDL._AC_UY218_.jpg
Rating(out of 5): No rating
------- END PRODUCT ----------


Product: Ziox Thunder Mega (Black-Grey)
Price: Available from different sellers
Image URL: https://m.media-amazon.com/images/I/41YHWMqaOKL._AC_UY218_.jpg
Rating(out of 5): 5.0
------- END PRODUCT ----------


Product: Nillkin Case for Apple iPhone 12 Pro Max (6.7" Inch) Aoge Leather 360 Protection Elite Business Case with Soft Microfiber Lining & Internal Card Slot Brown
Price: 2,499
Image URL: https://m.media-amazon.com/images/I/71EtsSDD0fL._AC_UY218_.jpg
Rating

In [20]:
def get_weather(url):
    headers = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}
    response = requests.post(url,headers=headers)
    soup = BeautifulSoup(response.content, "lxml")
    periods = soup.find_all("time-layout", {"summarization":["12hourly"]})[1].find_all("start-valid-time")
    short_descs = soup.find("weather").find_all("weather-conditions")
    max_temperatures = soup.find("temperature",{"type":["maximum"]}).find_all("value")
    min_temperatures = soup.find("temperature",{"type":["minimum"]}).find_all("value")
    descriprions = soup.find_all("parameters")[0].wordedforecast.find_all("text")
    # print(periods)
    i = 0
    for row in periods:
        print(f"""
        Period:            {periods[i].text}
        Short Description: {short_descs[2*i].get("weather-summary")}
        Temperature:       {max_temperatures[i].text}
        Description:       {descriprions[2*i].text}
        ---------------------------------------------------------------------------------------------------------------------------------
    """)
        i = i + 1

In [21]:
get_weather("https://forecast.weather.gov/MapClick.php?lat=37.7749&lon=-122.4194&unit=0&lg=english&FcstType=dwml")


        Period:            2021-05-21T23:00:00-07:00
        Short Description: Mostly Clear
        Temperature:       64
        Description:       Mostly clear, with a low around 48. Light and variable wind becoming southwest around 6 mph. 
        ---------------------------------------------------------------------------------------------------------------------------------
    

        Period:            2021-05-22T18:00:00-07:00
        Short Description: Mostly Clear and Breezy then Mostly Clear
        Temperature:       64
        Description:       Mostly clear, with a low around 50. Breezy, with a west southwest wind 18 to 23 mph decreasing to 11 to 16 mph after midnight. Winds could gust as high as 30 mph. 
        ---------------------------------------------------------------------------------------------------------------------------------
    

        Period:            2021-05-23T18:00:00-07:00
        Short Description: Mostly Clear
        Temperature:       68
 